

# Projeto 3 - Ciência dos dados 

###### Nomes: Gustavo, Matheus, Pedro Carani e Thiago David






# Modelo preditivo do indíce Ibovespa.

# O que é o Ibovespa?

 O Ibovespa é o principal indicador de desempenho das ações negociadas na bolsa de valores B3, composto pelas empresas mais importantes do mercado de capitais brasileiros, é um ótimo paramêtro da economia nacional.



 A pergunta que nosso modelo tenta responder é: 


# É possível prever o indíce baseado no passado? 

Para responder essa pergunta, vamos fazer um modelo preditivo utilizando a técnica de Regressão Linear.

## Regressão Linear

 A regressão linear é um método de ajuste de reta dos dados que estão sendo considerados, e pode ser calculado de acordo com a seguinte fórmula: 

$$y_i=\beta_0+\beta_1x_i+\epsilon_i$$

Também, ajuda na procura por padrões e anomalias desses dados.  Primeiro, para fazer uma regressão linear, é necessário achar os coeficientes $\beta_0$ e $\beta_1$.

Para achar o $\beta_0$ nós usamos a seguinte fórmula:

$$\beta_0=\bar{y}-\beta_1\bar{x}$$

E para achar o $\beta_1$ usamos a seguinte fórmula: $$\beta_1=\frac{S_{XY}}{S_{XX}}=\frac{\sum_{i=1}^n(x_i-\bar{x})(y_i-\bar{y})}{\sum_{i=1}^n(x_i-\bar{x})^2}$$

Após achar os coeficientes é necessário achar o vetor de resíduos através da fórmula: $\epsilon_i=y_i-(\beta_0+\beta_1x_i)$. Com os coeficientes e o vetor de resíduo já é possível a reta da regressão linear.
    

## Regressão Linear - Python

   Para podermos usar a regressão linear no nosso projeto usamos as seguintes bibliotecas: 

#### Importando bibliotecas: 

In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd
from scipy.stats import norm 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn import metrics
import random
np.random.seed(1)

## Sobre o nosso Modelo:
   Os dados foram coletados do site https://finance.yahoo.com/:

     - Ibovespa (2018-2019)
     
     
     
     
   

## Qual variável explicativa vai ser utilizada?

 A variável explicativa a ser utilzada no modelo, será os valores do índice ibovespa 1 passo atras, já que o modelo preditivo é baseado no passado do índice. Ou seja, o nosso X, vai ser equivalente ao Y(n-1), onde Y é a pontuacao do Ibovespa.


## Nosso modelo:

## Lendo os arquivos de dados:

In [2]:
ibov=pd.read_csv("bov.csv")

# Análise exploratória dos dados:

## Dados do Ibovespa:

In [3]:
ibov.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-11-08,87719.0,88570.0,85620.0,85620.0,85620.0,5297700
1,2018-11-09,85620.0,86233.0,84030.0,85641.0,85641.0,4814400
2,2018-11-12,85644.0,86227.0,85009.0,85525.0,85525.0,3534000
3,2018-11-13,85531.0,85941.0,84071.0,84914.0,84914.0,4410700
4,2018-11-14,84898.0,85973.0,84267.0,85973.0,85973.0,5591600


## Vamos analisar apenas os dados de fechamento

In [4]:
ibov["Close"].head()

0    85620.0
1    85641.0
2    85525.0
3    84914.0
4    85973.0
Name: Close, dtype: float64

## Criando um novo Dataset com apenas as informações necessárias (data e fechamento) para a análise:

In [5]:
ibov=pd.DataFrame(ibov, columns=['Date','Close'])


## Dataset do Ibovespa para o modelo:

In [6]:
ibov.head()

,Date,Close
0,2018-11-08,85620.0
1,2018-11-09,85641.0
2,2018-11-12,85525.0
3,2018-11-13,84914.0
4,2018-11-14,85973.0


## Separando os dados:

In [7]:
date=ibov['Date']
pontos=ibov['Close']


## Gráfico do Ibovespa durante os anos de 2018 e 2019.

In [ ]:
plt.figure(1, figsize=(10,6))
plt.plot(date, pontos)
plt.title('Ibov [2018 - 2019]', fontsize=16)
plt.xlabel('Dias', fontsize=14)
plt.ylabel('Pontos', fontsize=14)
plt.show();



## Boxplot do índice Ibovespa


In [ ]:
ibov.boxplot();


Nesse boxplot, é possível observar que, nesse período analisado, o indíce mantém a maioria dos seus valores entre 95.000 a 102.000 pontos, além disso, é possível observar a média do período, próximo a 97.000 pontos

## Histograma do Ibovespa

In [ ]:
ibov.hist();

 Pelo histograma, é possivel analisar que as maiorias dos dados se encontram entre 95.000 pontos a 102.000 pontos.

## Pontos importantes:

In [ ]:
ibov.describe()

## Ponto mínimo  

Este é o ponto mínimo do índice Ibovespa.

In [ ]:
ibov.min()

## Ponto máximo

Este é o ponto máximo do índice Ibovespa.

In [ ]:
ibov.max()

## Média

In [ ]:
ibov.mean()

## Modelo: 

## Primeiro, vamos fazer um modelo baseado apenas no índice anterior:

$$\bar{Y}=\beta_0 + \beta_1*Y[n-1]$$

### Criando um dataset com o Índice Ibov anterior:

In [ ]:
lista = np.array(pontos)
lista=lista[1:]
ibov_anterior=pd.DataFrame(lista)




ibov_novo=ibov.join(ibov_anterior, lsuffix='_caller', rsuffix='_other')
ibovespa=ibov_novo.rename(columns={"Close": "Fechamento anterior", 0: "Fechamento"})
ibovespa=ibovespa.dropna()
ibovespa.head()


In [ ]:
anterior=ibovespa["Fechamento anterior"]
fechamento=ibovespa["Fechamento"]

ibovespa.corr()

Como observado na matriz de correlacao acima, vemos que o retorno passado tem grande correlacao com o retorno presente.

### Separar dados para treinamento e teste

In [ ]:
train, test = train_test_split(ibovespa, test_size=0.2, random_state = 1)

### Treinamento

In [ ]:
X_train = np.array(train["Fechamento anterior"]).reshape(-1, 1)
y_train = train['Fechamento']

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_train)

In [ ]:
print("B0=",model.intercept_)
print("B1=",model.coef_)

Podemos observar que o coeficiente B1 deu muito proximo da correlação de ambas pontuacoes, tal fato pode ser explicado devido a correlacao estar proxima de 1, sendo assim, a variacao de uma unidade no regressor, impactaria numa variacao igual na variavel preditiva, fazendo com que o coeficiente B1 da regressao capte o mesmo efeito.

In [ ]:
plt.figure(1, figsize=(10,6))
plt.title('Regressão Linear  | Treinameto' )
plt.scatter(X_train, y_train, label='Pontuação')
plt.plot(X_train, y_pred, color='r', label='Regressão')
plt.xlabel('Pontuação passada')
plt.ylabel('Pontuação presente')
plt.legend()
plt.show()

#### Teste

In [ ]:
X_test = np.array(test["Fechamento anterior"]).reshape(-1, 1)
y_test = test['Fechamento']


In [ ]:
y_previsto=X_test*0.98965823+1109.8830689665774


In [ ]:
plt.figure(1, figsize=(10,6))
plt.title('Regressão Linear  | Teste' )
plt.scatter(X_test, y_test, label='Pontuação')
plt.plot(X_test, y_previsto, color='r', label='Regressão')
plt.xlabel('Pontuação passada')
plt.ylabel('Pontuação presente')
plt.legend()
plt.show()

### R² (modelo com um passo atrás)

In [ ]:
metrics.r2_score(y_train, y_pred)

Como o R² se aproximou de 100%, aproximadamente 95%, isto é um indicativo de que a variável preditiva, é explicada pelo regressor x, no caso, o índice defasado a um passo atrás.

## Agora, vamos fazer um modelo baseado no índice defasado respectivamente, de um a quatro passos atrás:

$$\bar{Y}=\beta_0 + \beta_1*Y[n-1] + \beta_2*Y[n-2] + \beta_3*Y[n-3] + \beta_4*Y[n-4]$$

In [ ]:
vals=fechamento.values
K = 5
n = len(vals)
x_dias = []
for k in range(K):
    x = vals[k:(n-K+k)]
    x_dias.append(x)

x_dias = np.array(x_dias).transpose()

ibov_anterior1=pd.DataFrame(x_dias)
X = x_dias[:, :-1]
y = x_dias[:, -1:]

Aqui é separado os dados de teste e os dados para treinamento, os dados dados foram separados na seguinte proporção: 20% para teste e 80% para treinamento


In [ ]:
random.seed( 30 )
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_train)

In [ ]:
B0=model.intercept_
B=model.coef_

print("B0=",model.intercept_)
print("B's=",model.coef_)


Podemos observar nesse modelo, que o índice defasado a três passos atrás, impacta negativamente no índice presente. Assim como, o índice defasado a um, dois e quatro passos atrás, demonstram estar relacionados positivamente com o índice presente.

In [ ]:
plt.figure(1, figsize=(10,6))
plt.title('Regressão Linear  | Treinameto' )
plt.scatter(X_train[:,3], y_train, label='Pontuação')
plt.plot(X_train[:,3], y_pred, color='r', label='Regressão')
plt.xlabel('Pontos')
plt.ylabel('Pontuação')
plt.legend()
plt.show()

In [ ]:
x=X_test

In [ ]:
y_previsao=[]
i=0
while i < len(x):
    y_previsao.append(2583.24420774+X_test[i][0]*0.00249712+X_test[i][1]*0.07289824+X_test[i][2]*-0.10062166+X_test[i][3]*0.99975053)
    i+=1
      


In [ ]:
plt.figure(1, figsize=(10,6))
plt.title('Regressão Linear  |  Teste')
plt.plot(X_test[:,3], y_previsao , color='r', label='Regressão')
plt.scatter(X_test[:,3], y_test, edgecolor='w', label='Pontos')
plt.xlabel('Pontos')
plt.ylabel('Pontos')

plt.show()

## R² (modelo de 1 a 4  passos atrás):

In [ ]:
metrics.r2_score(y_train, y_pred)

# Análise dos resíduos 

A Análise de Resíduos consiste em um conjunto de técnicas para investigar a adequabilidade do modelo com base nos resíduos.



In [ ]:
e=(y_previsto[1:]-y_test)**2


In [ ]:
from scipy.stats import norm

mu, std = norm.fit(e)
        
x = np.linspace(min(e), max(e), 100)
p = norm.pdf(x, mu, std)
plt.hist(e, density=True)

plt.plot(x, p)
plt.show()




Como podemos observar os resíduos não seguem uma distribuição normal, isso pode ser explicado pela aleatoriedade do índice.

In [ ]:
eb=e.mean()
eb=eb**(1/2)

# Conclusão:

$Yn=\beta_0+\beta_1Y(n-1)+\beta_2Y(n-2)+\beta_3Y(n-3)+\beta_4Y(n-4)$ + $\epsilon_i$

In [ ]:
y=fechamento
y=np.array(y)
pred=[]

len(y)

In [ ]:
n=0
while n < len(y)+1:
    if n>5:
        pred.append(1799.20843632+0.00249712*y[n-1]+0.07289824*y[n-2]-0.15793943*y[n-3]+0.99975053*y[n-4]+eb)
    n+=1


len(pred)


In [ ]:
plt.figure(1, figsize=(10,6))
plt.scatter(date[6:], pontos[6:])
plt.title('Ibov [2018 - 2019]', fontsize=16)
plt.plot(date[6:], pred, color='r', label='Regressão')
plt.xlabel('Dias', fontsize=14)
plt.ylabel('Pontos', fontsize=14)
plt.show();

 Como o mercado de ações é de certa maneira imprevisível, dificilmente o modelo funcionará perfeitamente, pois não é possível saber como o mercado reagirá numa possível catástrofe ecônomica, nem em momentos de euforia. No entanto, ao analisar o passado do índice, é possível concluir que ao fazer uma auto-regressão, o modelo se mostra até que bem.

Quando consultado a literatura de artigos financeiros, identifiquei que os mesmos seguem passeio aleatorio, logo, nao podem ser previstos.



Quando analisado o modelo de regressao de linear utilizado pelo modelo, esse se aproxima muito de um processo autoregressivo, o primeiro de ordem 1 e o segundo de ordem 4

## Referências:

    - Montgomery. Cap 11, pág 333.
    -https://scikit-learn.org/stable/modules/generated/sklearn.metrics.explained_variance_score.html#sklearn.metrics.explained_variance_score
    -https://scikit-learn.org/stable/modules/model_evaluation.html
    -http://www.b3.com.br/pt_br/
    -https://finance.yahoo.com/